### Initialization

In [1]:

import numpy as np
import matplotlib.pyplot as plt
import h5py
import pickle

import sys
sys.path.append('../common')
from utils import process_group

fpath1 = '/home/8ididata/2021-2/babnigg202107_2/cluster_results_QZ'
fpath2 ='/home/8ididata/2022-1/babnigg202203/cluster_results_QZ'


### Averaging of SA-XPCS results, temperature ramp up

In [2]:


avg_ramp_up, t_el, ql_dyn, ql_sta = process_group(group='B039',
                                              num_sections=10,
                                              prefix=fpath1, 
                                              zone_idx='auto',
                                              num_cores=24)


2023-12-20 16:26:31,431 utils                   : total number of files in B039  is 2780
2023-12-20 16:26:40,981 utils                   : label='B039_section_01': remove 31 datasets out of 278
2023-12-20 16:26:40,989 utils                   : label='B039_section_03': remove 32 datasets out of 278
2023-12-20 16:26:41,063 utils                   : label='B039_section_02': remove 28 datasets out of 278
2023-12-20 16:26:41,169 utils                   : label='B039_section_05': remove 33 datasets out of 278
2023-12-20 16:26:41,228 utils                   : label='B039_section_04': remove 32 datasets out of 278
2023-12-20 16:26:41,242 utils                   : label='B039_section_00': remove 28 datasets out of 278
2023-12-20 16:26:41,354 utils                   : label='B039_section_07': remove 32 datasets out of 278
2023-12-20 16:26:41,484 utils                   : label='B039_section_08': remove 30 datasets out of 278
2023-12-20 16:26:41,509 utils                   : label='B039_section_0

### Averaging of SA-XPCS results, temperature ramp down

In [3]:

avg_ramp_down, _, _, _ = process_group(group='B040',
                                 num_sections=10, 
                                 prefix=fpath1,
                                 zone_idx='auto',
                                 num_cores=24)


2023-12-20 16:26:42,501 utils                   : total number of files in B040  is 2780
2023-12-20 16:26:52,603 utils                   : label='B040_section_01': remove 34 datasets out of 278
2023-12-20 16:26:52,662 utils                   : label='B040_section_02': remove 31 datasets out of 278
2023-12-20 16:26:52,698 utils                   : label='B040_section_03': remove 31 datasets out of 278
2023-12-20 16:26:52,777 utils                   : label='B040_section_04': remove 32 datasets out of 278
2023-12-20 16:26:52,849 utils                   : label='B040_section_05': remove 31 datasets out of 278
2023-12-20 16:26:52,967 utils                   : label='B040_section_06': remove 31 datasets out of 278
2023-12-20 16:26:53,140 utils                   : label='B040_section_09': remove 30 datasets out of 278
2023-12-20 16:26:53,141 utils                   : label='B040_section_07': remove 31 datasets out of 278
2023-12-20 16:26:53,206 utils                   : label='B040_section_0

### Averaging of background 

In [4]:

avg_bg, _, _, _ = process_group(group='D029',
                                 num_sections=1, 
                                 prefix='/home/8ididata/2021-2/babnigg202107_2/cluster_results_QZ',
                                 zone_idx='auto',
                                 num_cores=24)


2023-12-20 16:26:54,262 utils                   : total number of files in D029  is 200
2023-12-20 16:26:58,710 utils                   : label='D029_section_00': remove 26 datasets out of 200


### Data reduction

Subtracting I(Q) with background, converting to absolute scattering cross-section, and dump to pickle file for plotting

In [7]:

fname = 'E0110_S3_4_Ramp_340C10p_att00_Rq0_00001_0001-100000.hdf'

with h5py.File(fpath2+'/'+fname, 'r') as f:
    _ = f['measurement']['sample']['absolute_cross_section_scale'][()]
    abs_coeff = _[0][0]

for ii in range(len(avg_ramp_up)):
    avg_ramp_up[ii]["saxs_1d"] = abs_coeff*(avg_ramp_up[ii]["saxs_1d"] - avg_bg[0]["saxs_1d"])

for ii in range(len(avg_ramp_down)):
    avg_ramp_down[ii]["saxs_1d"] = abs_coeff*(avg_ramp_down[ii]["saxs_1d"] - avg_bg[0]["saxs_1d"])


with open('checkpoint.pickle', 'wb') as f:  # 'wb' for binary write mode
    pickle.dump(avg_ramp_up, f)  # Dump the object to the file
    pickle.dump(avg_ramp_down, f)
    pickle.dump(t_el, f)
    pickle.dump(ql_sta, f)
    pickle.dump(ql_dyn, f)
